In [3]:
import pickle

import numpy as np
import keras
import keras.utils
from keras.preprocessing.text import Tokenizer


In [4]:
RHYME_MODEL_NAME = "RhymeModel"
SEQUENCE_MODEL_NAME = "StructureModel"
RHYME_MODEL = keras.models.load_model(f"data/models/{RHYME_MODEL_NAME}/weights.h5")
SEQUENCE_MODEL = keras.models.load_model(f"data/models/{SEQUENCE_MODEL_NAME}/weights.h5")

In [5]:
with open(f"data/models/{RHYME_MODEL_NAME}/tokenizer.pickle", 'rb') as handle:
    CHAR_TOKENIZER = pickle.load(handle)

with open(f"data/models/{SEQUENCE_MODEL_NAME}/tokenizer.pickle", 'rb') as handle:
    WORD_TOKENIZER = pickle.load(handle)


In [6]:
REVERSE_CHAR_TOKEN_INDEX = {v: k for k, v in CHAR_TOKENIZER.word_index.items()}

def generate_rhyme(model: keras.Model, tokenizer: Tokenizer, sequence_length: int, in_word: str, max_chars: int = 30):
    out_word = ''
    for i in range(max_chars):
        encoded = tokenizer.texts_to_sequences([f"{in_word} {out_word}"])[0]
        padded = keras.utils.pad_sequences([encoded], maxlen=sequence_length - 1, padding='pre')
        predicted = np.argmax(model.predict(padded, verbose=0))
        out_char = REVERSE_CHAR_TOKEN_INDEX.get(predicted, None)
        if out_char == " " or out_char is None:
            break
        out_word += out_char
    return out_word

In [7]:
REVERSE_WORD_TOKEN_INDEX = {v: k for k, v in WORD_TOKENIZER.word_index.items()}

def generate_line(model: keras.Model, tokenizer: Tokenizer, sequence_length: int, in_line: str, max_words: int = 60):
    out_line = ""
    for _ in range(max_words):
        encoded = tokenizer.texts_to_sequences([f"{in_line} \n {out_line}"])[0]
        padded = keras.utils.pad_sequences([encoded], maxlen=sequence_length - 1, padding='pre')
        predicted = np.argmax(model.predict(padded, verbose=0))
        out_word = REVERSE_WORD_TOKEN_INDEX.get(predicted, None)
        out_line += ' ' + out_word
        if out_word == '\n':
            break
    return out_line

In [10]:
SEEDS = [
    " Scares me a lot",
    " Looks like the tree",
    " I wake up in the morning"
]

SEED_LINE = SEEDS[1]

POEM = SEED_LINE + ' \n'

for i in range(1, 4):
    line = generate_line(SEQUENCE_MODEL, WORD_TOKENIZER, 43, POEM)
    if i % 2:
        poem_words = POEM.split(' ')
        line_words = line.split(' ')
        rhyme = generate_rhyme(RHYME_MODEL, CHAR_TOKENIZER, 62, poem_words[-2])
        line_words[-2] = rhyme
        line = ' '.join(line_words)
    POEM += line
print(f"SEED: \n{SEED_LINE}")
print()
print(f"POEM: \n{POEM}")

SEED: 
Looks like the tree

POEM: 
Looks like the tree 
 john bar dogmas seventy sea 
 also doctor need hook 
 she wire cook 

